In [10]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from dataset import ptb

기본적인 RNN 계층 구현

In [ ]:
class RNN:
  def __init__(self, Wx, Wh, b):
    self.params = [Wx, Wh, b]
    self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
    self.cache = None

  # RNN의 순전파 계산
  def forward(self, x, h_prev):
    Wx, Wh, b = self.params

    t = np.matmul(h_prev, Wh) + np.matmul(x, Wx) + b
    h_next = np.tanh(t)

    self.cache = (x, h_prev, h_next)
    return h_next

  # RNN의 역전파 계산
  def backward(self, dh_next):
    Wx, Wh, b = self.params
    x, h_prev, h_next = self.cache

    dt = dh_next * (1 - h_next ** 2)
    db = np.sum(dt, axis=0)
    dWh = np.matmul(h_prev.T, dt)
    dh_prev = np.matmul(dt, Wh.T)
    dWx = np.matmul(x.T, dt)
    dx = np.matmul(dt, Wx.T)

    self.grads[0][...] = dWx
    self.grads[1][...] = dWh
    self.grads[2][...] = db

    return dx, dh_prev

앞서 구현한 RNN 계층들을 모아서 처리하는 Time RNN을 구현해본다. Time RNN 계층은 RNN 계층 T개를 연결한 신경망이다.

In [24]:
class TimeRNN:
  def __init__(self, Wx, Wh, b, stateful=False):
    self.params = [Wx, Wh, b]
    self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)] # 저장할 미분값들을 0으로 초기화
    self.layers = None                                                    # 다수의 RNN 계층을 리스트로 저장하는 용도

    self.h, self.dh = None, None
    self.stateful = stateful

  def set_state(self, h):
    self.h = h

  def reset_state(self):
    self.h = None

  # Time RNN 순전파 계산
  def forward(self, xs):
    Wx, Wh, b = self.params
    N, T, D = xs.shape      # N:미니배치 크기, T:시계열 데이터의 분량, D:입력벡터의 차원 수
    D, H = Wx.shape         # D:입력벡터의 차원 수, H:은닉 벡터의 차원 수

    self.layers = []
    hs = np.empty((N, T, H), dtype='f')     # 출력값을 담을 float형 그릇, N:미니배치 크기, T:시계열 데이터의 분량, H:은닉 벡터의 차원 수

    if not self.stateful or self.h is None: # 처음 호출 시에 RNN계층의 은닉상태 h를 0행렬로 초기화
      self.h = np.zeros((N, H), dtype='f')

    for t in range(T):
      layer = RNN(*self.params)                   # 앞서 구현했던 RNN 계층을 이용
      self.h = layer.forward(xs[:, t, :], self.h) # t번째 입력에 대해서 RNN 계층 순전파 수행
      hs[:, t, :] = self.h                        # 각 시각 t의 출력(은닉 상태)을 저장
      self.layers.append(layer)                   # 생성하여 사용된 RNN 계층을 laysers 리스트에 추가하여 저장

    return hs

  # Time RNN 역전파
  def backward(self, dhs):
    Wx, Wh, b = self.params
    N, T, H = dhs.shape     # N:미니배치 크기, T:시계열 데이터의 분량, H:은닉 벡터의 차원 수
    D, H = Wx.shape         # D:입력벡터의 차원 수, H:은닉 벡터의 차원 수

    dxs = np.empty((N, T, D), dtype='f')  # 역전파 입력
    dh = 0                                # 역전파를 거치며 합산된 기울기
    grads = [0, 0, 0]                     # 미분값 초기화

    for t in reversed(range(T)): # 역순서의 인덱스로 순회
      layer = self.layers[t]                    # RNN 계층을 역으로 순회
      dx, dh = layer.backward(dhs[:, t, :] + dh) # 역전파 수행 (순전파에서 분기되었으므로 역전파에서는 기울기를 합산, 분기의 반대는 합산이다!)
      dxs[:, t, :] = dx                          # 각 시각 t에서 얻은 x 기울기를 저장

      # t를 순회할 때마다 역전파로 얻은 가중치를 위 로컬변수로 초기화 한 grads에 합산
      for i, grad in enumerate(layer.grads):
        grads[i] += grad

    # 합산된 가중치 기울기의 최종 결과를 멤버변수 self.grads에 덮어씀
    for i, grad in enumerate(grads):
      self.grads[i][...] = grad
    self.dh = dh

    return dxs

RNN을 사용하여 언어 모델 'RNNLM'을 구현해본다.
이때 Embedding 층을 한꺼번에 처리하는 Time Embedding 층과, Affine 층을 한꺼번에 처리하는 Time Affine 층이 필요하다.

RRNLM 구현에 필요한 Time Embedding 층을 구현한다.

In [5]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None

    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out

    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        np.add.at(dW, self.idx, dout)
        return None

class TimeEmbedding:
  def __init__(self, W):
    self.params = [W]
    self.grads = [np.zeros_like(W)]
    self.layers = None
    self.W = W

  def forward(self, xs):
    N, T = xs.shape
    V, D = self.W.shape

    out = np.empty((N, T, D), dtype='f')
    self.layers = []

    for t in range(T):
      layer = Embedding(self.W)
      out[:, t, :] = layer.forward(xs[:, t])
      self.layers.append(layer)

    return out

  def backward(self, dout):
    N, T, D = dout.shape

    grad = 0
    for t in range(T):
      layer = self.layers[t]
      layer.backward(dout[:, t, :])
      grad += layer.grads[0]

    self.grads[0][...] = grad
    return None


RNNLM 구현에 필요한 Time Affine 층을 구현한다.
Affine 계층은 어파인 변환(순전파 때 행렬의 곱을 계산)을 수행하는 계층이다.

[유튜브-한경훈] https://www.youtube.com/watch?v=QIi8-4uea7M

[활성화 함수, Affine, Softmax 계층] https://deep-learning-study.tistory.com/18

[Affine, softmax, backprop] https://m.blog.naver.com/fbfbf1/222424511374

In [6]:
class Affine:
    def __init__(self, W, b):
        self.params = [W, b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None

    def forward(self, x):
        W, b = self.params
        out = np.dot(x, W) + b
        self.x = x
        return out

    def backward(self, dout):
        W, b = self.params
        dx = np.dot(dout, W.T)
        dW = np.dot(self.x.T, dout)
        db = np.sum(dout, axis=0)

        self.grads[0][...] = dW
        self.grads[1][...] = db
        return dx

class TimeAffine:
  def __init__(self, W, b):
    self.params = [W, b]
    self.grads = [np.zeros_like(W), np.zeros_like(b)] # 가중치와 편향의 기울기를 저장
    self.x = None

  def forward(self, x):
    N, T, D = x.shape
    W, b = self.params

    rx = x.reshape(N*T, -1)
    out = np.dot(rx, W) + b
    self.x = x
    return out.reshape(N, T, -1)

  def backward(self, dout):
    x = self.x
    N, T, D = x.shape
    W, b = self.params

    dout = dout.reshape(N*T, -1)
    rx = x.reshape(N*T, -1)

    db = np.sum(dout, axis=0)
    dW = np.dot(rx.T, dout)
    dx = np.dot(dout, W.T)
    dx = dx.reshape(*x.shape)

    self.grads[0][...] = dW
    self.grads[1][...] = db

    return dx

Softmax 계층을 구현할 때에는 손실 오차를 구하는 Cross Entropy Error 계층을 함께 구현한다. 여기에서는 모든 산출된 손실을 합산하여 평균한 값을 최종 손실로 산출한다.

In [22]:
class TimeSoftmaxWithLoss:
  def __init__(self):
    self.params, self.grads = [], []
    self.cache = None
    self.ignore_label = -1


  def softmax(self, x):
    if x.ndim == 2:
        x = x - x.max(axis=1, keepdims=True)
        x = np.exp(x)
        x /= x.sum(axis=1, keepdims=True)
    elif x.ndim == 1:
        x = x - np.max(x)
        x = np.exp(x) / np.sum(np.exp(x))

    return x

  def forward(self, xs, ts): # xs : 전달받는 점수(확률로 정규화되기 전의 값), ts 정답 레이블
    N, T, V = xs.shape

    if ts.ndim == 3: # 정답 레이블이 원핫 벡터인 경우
      ts = ts.argmax(axis=2)

    mask = (ts != self.ignore_label)

    # 배치용과 시계열용을 정리(reshape)
    xs = xs.reshape(N * T, V)
    ts = ts.reshape(N * T)
    mask = mask.reshape(N * T)

    ys = self.softmax(xs)
    ls = np.log(ys[np.arange(N * T), ts])
    ls *= mask                # ignore_label에 해당하는 데이터는 손실을 0으로 설정
    loss = -np.sum(ls)
    loss /= mask.sum()        # loss 값들을 합산하고 평균을 낸다

    self.cache = (ts, ys, mask, (N, T, V))
    return loss

  def backward(self, dout=1):
    ts, ys, mask, (N, T, V) = self.cache

    dx = ys
    dx[np.arange(N * T), ts] -= 1
    dx *= dout
    dx /= mask.sum()
    dx *= mask[:, np.newaxis]  # ignore_label에 해당하는 데이터는 기울기를 0으로 설정

    dx = dx.reshape((N, T, V))

    return dx


RNNLM 구현

In [18]:
sys.path.append('..')

class SimpleRnnlm:
  def __init__(self, vocab_size, wordvec_size, hidden_size):
    V, D, H = vocab_size, wordvec_size, hidden_size
    rn = np.random.randn

    # 가중치 초기화
    embed_W = (rn(V, D) / 100).astype('f')
    rnn_Wx = (rn(D, H) / np.sqrt(D)).astype('f')    # Xavier 초기값
    rnn_Wh = (rn(H, H) / np.sqrt(H)).astype('f')    # Xavier 초기값

    rnn_b = np.zeros(H).astype('f')
    affine_W = (rn(H, V) / np.sqrt(H)).astype('f')  # Xavier 초기값
    affine_b = np.zeros(V).astype('f')

    # 계층 생성
    self.layers = [
        TimeEmbedding(embed_W),
        TimeRNN(rnn_Wx, rnn_Wh, rnn_b, stateful=True),
        TimeAffine(affine_W, affine_b)
    ]
    self.loss_layer = TimeSoftmaxWithLoss()
    self.rnn_layer = self.layers[1]

    # 모든 가중치와 기울기를 리스트에 모은다.
    self.params, self.grads = [], []
    for layer in self.layers:
      self.params += layer.params
      self.grads += layer.grads

  def forward(self, xs, ts):
    for layer in self.layers:
      xs = layer.forward(xs)

    loss = self.loss_layer.forward(xs, ts)
    return loss

  def backward(self, dout=1):
    dout = self.loss_layer.backward(dout)
    for layer in reversed(self.layers):
      dout = layer.backward(dout)
    return dout

  def reset_state(self):
    self.rnn_layer.reset_state()

RNNLM을 활용한 학습코드

In [19]:
# 사용할 옵티마이저는 아래 SGD(Stochastic Gradient Descent)
class SGD:
    '''
    확률적 경사하강법(Stochastic Gradient Descent)
    '''
    def __init__(self, lr=0.01):
        self.lr = lr

    def update(self, params, grads):
        for i in range(len(params)):
            params[i] -= self.lr * grads[i]

In [25]:
import sys
sys.path.append('.')

# 하이퍼파라미터 설정
batch_size = 10     # 배치 크기
wordvec_size = 100  # 단어벡터 개수 (단어 사전 내 단어 수)
hidden_size = 100   # 은닉 상태 벡터의 원소 수
time_size = 5       # Truncated BPTT가 한 번에 펼치는 시간 크기
lr = 0.1
max_epoch = 100

# 학습 데이터 읽기(전체 중 1000개만)
corpus, word_to_id, id_to_word = ptb.load_data('train')
corpus_size = 1000  # 가져올 단어 수
corpus = corpus[:corpus_size]
vocab_size = int(max(corpus) + 1)

xs = corpus[:-1]
ts = corpus[1:]
data_size = len(xs)
print('corpus size : %d, vocabulary size : %d' % (corpus_size, vocab_size))

# 학습 시 사용하는 변수
max_iters = data_size // (batch_size * time_size)
time_idx = 0
total_loss = 0
loss_count = 0
ppl_list = []

# 모델 생성
model = SimpleRnnlm(vocab_size, wordvec_size, hidden_size)

# 사용할 옵티마이저 생성
optimizer = SGD(lr)

# 각 미니배치에서 샘플을 읽기 시작하는 위치를 계산하여 offsets에 저장
jump = (corpus_size - 1) // batch_size
offsets = [i * jump for i in range(batch_size)]

for epoch in range(max_epoch):
  for iter in range(max_iters):
    # 미니배치 획득
    batch_x = np.empty((batch_size, time_size), dtype='i')
    batch_t = np.empty((batch_size, time_size), dtype='i')
    for t in range(time_size):
      for i, offset in enumerate(offsets):
        batch_x[i, t] = xs[(offset + time_idx) % data_size]
        batch_t[i, t] = ts[(offset + time_idx) % data_size]
      time_idx += 1

    # 기울기를 구하여 매개변수 갱신
    # 순전파 수행하여 손실 산출
    loss = model.forward(batch_x, batch_t)
    # 역전파 수행하여 기울기 계산
    model.backward()
    # 모델의 가중치 업데이트
    optimizer.update(model.params, model.grads)

    total_loss += loss
    loss_count += 1

  # 에폭마다 퍼플렉서티(혼란도, 분기 수) 평가
  ppl = np.exp(total_loss / loss_count) # perplexity 계산
  print('| epoch %d | 퍼플렉서티 %.2f' % (epoch+1, ppl))
  ppl_list.append(float(ppl))
  total_loss, loss_count = 0, 0

corpus size : 1000, vocabulary size : 418
| epoch 1 | 퍼플렉서티 390.96
| epoch 2 | 퍼플렉서티 258.24
| epoch 3 | 퍼플렉서티 222.80
| epoch 4 | 퍼플렉서티 215.58
| epoch 5 | 퍼플렉서티 207.39
| epoch 6 | 퍼플렉서티 202.99
| epoch 7 | 퍼플렉서티 198.83
| epoch 8 | 퍼플렉서티 196.46
| epoch 9 | 퍼플렉서티 192.23
| epoch 10 | 퍼플렉서티 193.86
| epoch 11 | 퍼플렉서티 189.56
| epoch 12 | 퍼플렉서티 191.67
| epoch 13 | 퍼플렉서티 190.61
| epoch 14 | 퍼플렉서티 190.78
| epoch 15 | 퍼플렉서티 189.04
| epoch 16 | 퍼플렉서티 186.44
| epoch 17 | 퍼플렉서티 183.53
| epoch 18 | 퍼플렉서티 180.56
| epoch 19 | 퍼플렉서티 181.38
| epoch 20 | 퍼플렉서티 183.29
| epoch 21 | 퍼플렉서티 180.16
| epoch 22 | 퍼플렉서티 175.68
| epoch 23 | 퍼플렉서티 173.65
| epoch 24 | 퍼플렉서티 175.23
| epoch 25 | 퍼플렉서티 174.26
| epoch 26 | 퍼플렉서티 172.78
| epoch 27 | 퍼플렉서티 166.49
| epoch 28 | 퍼플렉서티 164.87
| epoch 29 | 퍼플렉서티 162.20
| epoch 30 | 퍼플렉서티 158.14
| epoch 31 | 퍼플렉서티 158.44
| epoch 32 | 퍼플렉서티 152.36
| epoch 33 | 퍼플렉서티 155.60
| epoch 34 | 퍼플렉서티 150.08
| epoch 35 | 퍼플렉서티 146.38
| epoch 36 | 퍼플렉서티 139.77
| epoch 37 | 퍼플렉서티 137.00
| epo